FM/LM connectivity score is measured by: 

- micromobility trip origin/destination counts (scooter, microtransit, bicycle) within 100ft buffer zone at the grouped bus stops.  
- Block level FMLM gap score 


## 1. read and merge scooter  & microtransit data 

In [11]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

df=pd.read_excel('data/Copy of Bird Raw Ride Data 01_01_22-12_31_22.xlsx')

crs={'init':'epsg:4326'}
geometry=[Point(xy) for xy in zip(df['start_lon'],df['start_lat'])]
scooter1=gpd.GeoDataFrame(df,crs=crs,geometry=geometry)

geometry=[Point(xy) for xy in zip(df['end_lon'],df['end_lat'])]
scooter2=gpd.GeoDataFrame(df,crs=crs,geometry=geometry)

df=pd.read_csv('data/Veo_Rides.csv')

crs={'init':'epsg:4326'}
geometry=[Point(xy) for xy in zip(df['LNG_PICKUP'],df['LAT_PICKUP'])]
scooter3=gpd.GeoDataFrame(df,crs=crs,geometry=geometry)


crs={'init':'epsg:4326'}
geometry=[Point(xy) for xy in zip(df['LNG_DROPOFF'],df['LAT_DROPOFF'])]
scooter4=gpd.GeoDataFrame(df,crs=crs,geometry=geometry)

scooter11=scooter1[['end_lat','end_lon']].rename(columns={'end_lat':'lat','end_lon':'lon'})
scooter22=scooter2[['start_lat','start_lon']].rename(columns={'start_lat':'lat','start_lon':'lon'})
scooter33=scooter3[['LAT_PICKUP','LNG_PICKUP']].rename(columns={'LAT_PICKUP':'lat','LNG_PICKUP':'lon'})
scooter44=scooter4[['LAT_DROPOFF', 'LNG_DROPOFF']].rename(columns={'LAT_DROPOFF':'lat','LNG_DROPOFF':'lon'})

dfs=[scooter11,scooter22,scooter33, scooter44]

scooter_df=pd.DataFrame()

scooter_df[['lat','lon']]=gpd.pd.concat([df[['lat','lon']] for df in dfs])

crs={'init':'epsg:4326'}
geometry=[Point(xy) for xy in zip(scooter_df['lon'],scooter_df['lat'])]
scooter_merge=gpd.GeoDataFrame(scooter_df,crs=crs,geometry=geometry)

scooter_merge.to_file('process/scooter_merge.shp')

C:\Users\anranzheng\Anaconda3\envs\geo\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\anranzheng\Anaconda3\envs\geo\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\anranzheng\Anaconda3\envs\geo\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initializa

In [12]:
df=pd.read_excel('data/Microtransit O-D Data_08152022_01012023.xlsx')

crs={'init':'epsg:4326'}
geometry=[Point(xy) for xy in zip(df['origin_longitude'],df['origin_latitude'])]
mcirotransit1=gpd.GeoDataFrame(df,crs=crs,geometry=geometry)

geometry=[Point(xy) for xy in zip(df['destination_longitude'],df['destination_latitude'])]
mcirotransit2=gpd.GeoDataFrame(df,crs=crs,geometry=geometry)

mcirotransit11=mcirotransit1[['origin_latitude','origin_longitude']].rename(columns={'origin_latitude':'lat','origin_longitude':'lon'})
mcirotransit22=mcirotransit2[['destination_latitude','destination_longitude']].rename(columns={'destination_latitude':'lat','destination_longitude':'lon'})

dfs=[mcirotransit11,mcirotransit22]

mcirotransit_df=pd.DataFrame()

mcirotransit_df[['lat','lon']]=gpd.pd.concat([df[['lat','lon']] for df in dfs])

crs={'init':'epsg:4326'}
geometry=[Point(xy) for xy in zip(mcirotransit_df['lon'],mcirotransit_df['lat'])]
mcirotransit_merge=gpd.GeoDataFrame(mcirotransit_df,crs=crs,geometry=geometry)

mcirotransit_merge.to_file('process/mcirotransit_merge.shp')

C:\Users\anranzheng\Anaconda3\envs\geo\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\anranzheng\Anaconda3\envs\geo\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\anranzheng\Anaconda3\envs\geo\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initializa

## 2. Spatial join the scooter and microtransit count with spatial unit in step 1 (in ArcGIS)

## 3. Calculate the block level FMLM gap score

In [21]:
origin=gpd.read_file('process/fmlm_score.shp')

1. Calculate the number of jobs + total population of each block centroid 


In [22]:
#calculate the average 
origin['job_pop']=origin['C000']+origin['totalPp']

2.
Find the distance to the nearest bus stop

Recode the distance
<0.25 mile: 0 ;
0.25-0.5 mile: 1 ;
0.5-0.75 mile: 2 ;
0.75-1 mile: 3


In [25]:
origin.NEAR_DIST=origin.NEAR_DIST*100/1.61

In [28]:
def assign_value(origin):

    if origin.NEAR_DIST <= 0.25:
        return 0
    elif origin.NEAR_DIST > 0.25 and  origin.NEAR_DIST <= 0.5:
        return 1
    elif origin.NEAR_DIST > 0.5 and  origin.NEAR_DIST <= 0.75:
        return 2
    else:
        return 3

# Apply the 'assign_value' function to each row of the DataFrame
origin['recode'] = origin.apply(assign_value, axis=1)


3. (number of jobs + total population)  * nearest distance to get the FMLM score at centroid level


In [30]:
origin['fmlm_score']=origin['recode']*origin['job_pop']

4. Aggregated the average values to the spatial unit


In [33]:
mean_score=origin.groupby(['index'])['fmlm_score'].mean().reset_index()

5. merge with micromobility trip count

In [36]:
trip= gpd.read_file('process/transit.shp')

trip=trip.rename(columns={'Join_Count':'inner_scoo','Join_Cou_1':'inner_micr'})

new_df=trip.merge(mean_score, how='left', on='index')

new_df.to_file('process/result2.shp')

In [43]:
new_df

,inner_scoo,TARGET_FID,inner_micr,TARGET_F_1,Route,total_trip,Total_Acti,W_C_TOTAL,BIKE_TOTAL,index,lat,lon,lat_1,lon_1,geometry,fmlm_score
0,11325,0,0,0,1.0,150.0,68.0,55.0,33.0,0,29.629470,-82.368050,0.0,0.0,"POLYGON ((-82.37880 29.63971, -82.37763 29.639...",29.082474
1,104852,1,0,1,9.0,1074.0,312.0,0.0,0.0,1,29.639612,-82.341515,0.0,0.0,"POLYGON ((-82.34559 29.65267, -82.34442 29.652...",0.000000
2,124533,2,0,2,8.0,869.0,307.0,0.0,0.0,2,29.639612,-82.341515,0.0,0.0,"POLYGON ((-82.34274 29.65368, -82.34157 29.653...",1.487685
3,68194,3,3338,3,14.0,720.0,3265.0,4325.0,1681.0,3,29.650175,-82.330467,0.0,0.0,"POLYGON ((-82.32270 29.66000, -82.32153 29.659...",1.576233
4,155560,4,5,4,3.0,272.0,33.0,0.0,0.0,4,29.653282,-82.338799,0.0,0.0,"POLYGON ((-82.33943 29.65833, -82.33826 29.658...",1.139394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,5,624,0,624,2.0,36.0,4.0,0.0,2.0,624,29.634177,-82.413232,0.0,0.0,"POLYGON ((-82.42305 29.61599, -82.42421 29.615...",49.767123
625,0,625,0,625,2.0,24.0,2.0,0.0,0.0,625,0.000000,0.000000,0.0,0.0,"POLYGON ((-82.44091 29.65987, -82.44207 29.659...",138.023256
626,0,626,0,626,2.0,24.0,0.0,0.0,0.0,626,0.000000,0.000000,0.0,0.0,"POLYGON ((-82.44579 29.65807, -82.44696 29.658...",161.157895
627,0,627,0,627,2.0,24.0,0.0,0.0,0.0,627,0.000000,0.000000,0.0,0.0,"POLYGON ((-82.44591 29.64726, -82.44708 29.647...",129.276596
